In [18]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Determine where the visualization will be rendered
output_notebook()

Loading BokehJS ...

In [19]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_flc_all_stats_postflash.pkl')
postflash_data

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
ic6de2u9q,ic6de2u9q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-03-09,13:59:17,B,12,4.7,LOW,True,0.5,7.312437,7.048593,6.917119,6.917119,17.943027,2.911503,1.499701,2013-03-09 13:59:17
icha24imq,icha24imq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-06-29,04:16:36,A,3,1.2,LOW,True,0.5,2.170188,1.976714,1.933412,1.933412,15.109935,2.360522,1.647262,2014-06-29 04:16:36
idp530aiq,idp530aiq_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-06-22,04:55:12,B,15,0.2,MED,True,0.5,8.972477,8.680973,8.854218,8.854218,22.985115,3.162331,43.404864,2018-06-22 04:55:12
ieg845feq,ieg845feq_flc.fits,16398,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-12-14,20:55:49,B,7310,100.0,MED,False,0.5,4685.206055,4683.406877,4759.056641,4759.056641,436.754791,419.022524,46.834069,2020-12-14 20:55:49
icrz19f7q,icrz19f7q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-05-25,22:31:24,B,12,4.7,LOW,True,0.5,7.337009,7.018181,6.969007,6.969007,27.715347,2.952669,1.493230,2015-05-25 22:31:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
icrz24ubq,icrz24ubq_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-06-23,19:35:12,B,5,2.0,LOW,True,0.5,3.178271,2.902492,2.999795,2.999795,20.268372,2.383841,1.451246,2015-06-23 19:35:12
ieg819ldq,ieg819ldq_flc.fits,16398,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2021-03-23,16:48:36,A,7310,100.0,MED,False,0.5,4864.202148,4861.856285,4934.516602,4934.516602,449.005798,429.721604,48.618563,2021-03-23 16:48:36
ic6de3riq,ic6de3riq_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-03-09,05:53:22,B,22,0.3,MED,True,0.5,13.063131,12.872396,12.798467,12.798467,11.174848,3.513827,42.907987,2013-03-09 05:53:22


In [20]:
postflash_data['norm_by_dur'] = (postflash_data['sig_mean']/postflash_data['flash_dur'])

In [21]:
test = fits.open('/Users/cmartlin/Desktop/Postflash_2021/mastDownload/HST/ic6de2u9q/ic6de2u9q_flc.fits')
test[0].header['FLASHDUR']
test.close()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/cmartlin/Desktop/Postflash_2021/mastDownload/HST/ic6de2u9q/ic6de2u9q_flc.fits'

In [22]:
pwd

'/System/Volumes/Data/grp/hst/wfc3u/final_postflash_2021/notebooks'

In [23]:
low_sub_pf_A_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 3)]

In [24]:
low_sub_pf_A_lvl_3_2021  = low_sub_pf_A_lvl_3[low_sub_pf_A_lvl_3.obsdate.str.contains("2021")]
low_sub_pf_A_lvl_3_2019  = low_sub_pf_A_lvl_3[low_sub_pf_A_lvl_3.obsdate.str.contains("2019")]

In [25]:
path_2021 = low_sub_pf_A_lvl_3_2021['path'][0]
print(path_2021)
test = fits.open('/Users/cmartlin/Desktop/Postflash_2021/mastDownload/HST/ieg815u3q/ieg815u3q_flc.fits')
print(test[0].header['FLASHDUR'])
print(test[0].header['FLASHLVL'])
test.close()

/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/ieg827g4q/ieg827g4q_flc.fits


FileNotFoundError: [Errno 2] No such file or directory: '/Users/cmartlin/Desktop/Postflash_2021/mastDownload/HST/ieg815u3q/ieg815u3q_flc.fits'

In [30]:
path_2019 = low_sub_pf_A_lvl_3_2019['path'][0]
print(path_2019)
test = fits.open('/Users/cmartlin/Desktop/Postflash_2021/mastDownload/HST/idus27w8q/idus27w8q_flc.fits')
print(test[0].header['FLASHDUR'])
print(test[0].header['FLASHLVL'])
test.close()

/grp/hst/wfc3u/final_postflash_2021/mastDownload/HST/idus27w8q/idus27w8q_flc.fits


FileNotFoundError: [Errno 2] No such file or directory: '/Users/cmartlin/Desktop/Postflash_2021/mastDownload/HST/idus27w8q/idus27w8q_flc.fits'

In [27]:
low_sub_pf_A_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 3)]

In [28]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_3['obsdate'] + ' ' + low_sub_pf_A_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_3['norm_by_dur'])
y = low_sub_pf_A_lvl_3['norm_by_dur']/low_sub_pf_A_lvl_3['norm_by_dur'][first_rootname]

print(mean_mean)

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
#p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 3 Shutter A")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

1.6265594636115062
-3.4492398520047405e-06


In [29]:
low_sub_pf_A_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 12)]
low_sub_pf_B_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 12)]



# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_12['obsdate'] + ' ' + low_sub_pf_B_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_12['norm_by_dur'])
y = low_sub_pf_B_lvl_12['norm_by_dur']/low_sub_pf_B_lvl_12['norm_by_dur'][first_rootname]

print(mean_mean)

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
#p.y_range=Range1d(0.9, 1.1)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 12 Shutter B")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

1.4912674007463798
Slope:  -5.69377315473145e-06
